# Example usage

## Initialize a GeslaDataset object
Place the `gesla.py` file in your working directory (or elsewhere on your path), and import the `GeslaDataset` class. Selecting and loading data files requires paths to the metadata .csv file and the directory containing the data files. Initialize a `GeslaDataset` object with these paths as follows.

In [12]:
import os
from gesla import GeslaDataset

data_path = os.path.join(os.path.expanduser('~'), 'Data', 'GESLA', 'GESLA3', '') # "../GESLAv3/GESLA3/"
meta_file = os.path.join(data_path, 'GESLA3_ALL 2.csv') # "../GESLAv3/GESLA3_ALL.csv"

g3 = GeslaDataset(meta_file=meta_file, data_path=data_path)
print("Done instantiating Data Loader")

# see here for GESLA map: https://gesla787883612.wordpress.com/

Done instantiating Data Loader


## Load data from a single file
If you want to work with data from a single record, and you know the filename you want, use the function `file_to_pandas` as follows. The function returns a `pandas.DataFrame` with data and flags, and a `pandas.Series` containing metadata.

In [13]:
filename = "acajutla-082c-slv-uhslc"
data, meta = g3.file_to_pandas(filename)
data

sea_level  qc_flag  use_flag
date_time                                        
2010-08-19 21:00:00      1.064        1         1
2010-08-19 22:00:00      0.905        1         1
2010-08-19 23:00:00      0.871        1         1
2010-08-20 00:00:00      0.998        1         1
2010-08-20 01:00:00      1.198        1         1
...                        ...      ...       ...
2018-12-31 19:00:00      1.563        1         1
2018-12-31 20:00:00      1.246        1         1
2018-12-31 21:00:00      1.001        1         1
2018-12-31 22:00:00      0.928        1         1
2018-12-31 23:00:00      1.007        1         1

[73347 rows x 3 columns]

## Load data from a list of files
If you want to work with data from multiple files, and you know the filenames you want, use the function `files_to_xarray` as follows. The function returns a `xarray.Dataset` object containing data, flags, and metadata.

In [14]:
filenames = [
    "abrams_river-380-can-meds",
    "acajutla-082c-slv-uhslc", 
    "yoshioka-hd26-jpn-jodc_jcg", 
    "west_point_a_la_hache-8761494-usa-noaa",
]
xr_dataset = g3.files_to_xarray(filenames)
print(xr_dataset)

<xarray.Dataset>
Dimensions:                  (date_time: 289251, station: 4)
Coordinates:
  * date_time                (date_time) datetime64[ns] 1971-06-09T05:00:00 ....
  * station                  (station) int64 0 1 2 3
Data variables: (12/27)
    sea_level                (station, date_time) float64 2.29 1.79 ... nan nan
    qc_flag                  (station, date_time) float64 1.0 1.0 ... nan nan
    use_flag                 (station, date_time) float64 1.0 1.0 ... nan nan
    filename                 (station) object 'acajutla-082c-slv-uhslc' ... '...
    site_name                (station) object 'Acajutla' ... 'Abrams_River'
    site_code                (station) object '082C' 'HD26' '8761494' '380'
    ...                       ...
    datum_information        (station) object 'Unspecified' ... 'Chart Datum ...
    instrument               (station) object 'Unspecified' ... 'Unspecified'
    precision                (station) object 'Unspecified' ... 'Unspecified'
    null_va

## Load data from the N closest records to a lat/lon location
Load data from records close to a particular location using the function `load_N_closest` as follows. Provide a lat/lon location and the number of desired records. The function returns a `xarray.Dataset` object containing data, flags, and metadata.  

Note the `UserWarning` that occurs when duplicate timestamps are encountered. The function `file_to_pandas` used to read each individual file keeps only the first of any duplicate timestamps.

In [15]:
data = g3.load_N_closest(lat=43.83, lon=-65.95, N=10)
print(data.site_name.values)

/home/pwjebel/Projects/coastalFlood/util/GeslaDataset/gesla.py:65: UserWarning: Duplicate timestamps in file yarmouth-365-can-meds were removed.
  warnings.warn(


['Yarmouth' 'Lower_Wedgeport' 'Pinkney_Point' 'Abbotts_Harbour'
 'Clarks_Harbour' 'Woods_Harbour' 'Tusket' 'Abrams_River' 'Flat_Island'
 'Wedgeport']


## Load data from the records in a lat/lon range
Load data from records in a rectangular lat/lon range using the function `load_lat_lon_range` as follows. Provide lat/lon extents of the range. The function returns a `xarray.Dataset` object containing data, flags, and metadata.

In [16]:
south_lat = 15
north_lat = 30
west_lon = -180
east_lon = -140

data = g3.load_lat_lon_range(
    south_lat=south_lat,
    north_lat=north_lat,
    west_lon=west_lon,
    east_lon=east_lon,
)
print(data.site_name.values)

['Honolulu_Hawaii' 'Hilo_Hawaii' 'Midway' 'Johnston' 'Kahului'
 'Nawiliwili' 'Mokuoloe' 'French_Frigate' 'Kawaihae' 'French_Frigate'
 'Barbers_Point_HI' 'Honolulu_Kewalo' 'Port_Allen' 'Kaumalapau_HI'
 'Honolulu_Hawaii' 'Honolulu_Pier_45' 'Honolulu' 'Hilo' 'Nawiliwili'
 'Sand_Island' 'Kahului' 'Mokuoloe' 'Kawaihae' 'Johnston_Atoll'
 'Port_Allen' 'Kaumalapau_Harbor' 'Kaunakakai_Harbor' 'Laiemaloo'
 'Fort_Kamehameha' 'Ford_Island']
